In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

22/03/02 12:44:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [50]:
#!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-01 14:33:33--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.12.20
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.12.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv.2’

fhvhv_tripdata_2021 100%[===================>] 699.83M  35.7MB/s    in 21s     

2022-03-01 14:33:54 (33.4 MB/s) - ‘fhvhv_tripdata_2021-02.csv.2’ saved [733822658/733822658]



In [3]:
!head fhvhv_tripdata_2021-02.csv

In [4]:
import pandas as pd

In [5]:
from pyspark.sql import types

In [6]:
df = spark.read\
.option("header", "true") \
.csv('fhvhv_tripdata_2021-02.csv')

In [7]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [8]:
df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('pickup_datetime', 'string'),
 ('dropoff_datetime', 'string'),
 ('PULocationID', 'string'),
 ('DOLocationID', 'string'),
 ('SR_Flag', 'string')]

In [9]:
homework_schema = types.StructType([
    types.StructField("hvfhs_license_num", types.StringType(), True),
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropoff_datetime", types.TimestampType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("SR_Flag", types.DoubleType(), True),
])

In [10]:
df = spark.read\
.option("header", "true") \
.schema(homework_schema)\
.csv('fhvhv_tripdata_2021-02.csv')

In [11]:
df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('SR_Flag', 'double')]

In [12]:
df=df.repartition(24)

In [23]:
df.write.parquet('fhvhv_tripdata_2021-02')

In [13]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [14]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02765|2021-02-05 15:13:57|2021-02-05 15:30:25|          71|          89|   null|
|           HV0003|              B02866|2021-02-04 11:40:47|2021-02-04 11:46:35|          20|         174|   null|
|           HV0003|              B02872|2021-02-02 13:21:54|2021-02-02 13:36:47|          78|         247|   null|
|           HV0003|              B02872|2021-02-03 22:05:16|2021-02-03 22:14:52|          75|         262|   null|
|           HV0003|              B02888|2021-02-02 16:41:27|2021-02-02 17:06:27|         138|          88|   null|
|           HV0003|              B02866|2021-02-06 03:00:04|2021-02-06 03:10:29|

In [22]:
df.registerTempTable('Amine_data_homework')

In [16]:
from pyspark.sql import functions as F

In [17]:
timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"

In [18]:
timeDiff = (F.unix_timestamp('dropoff_datetime', format=timeFmt)
            - F.unix_timestamp('pickup_datetime', format=timeFmt))

In [19]:
df = df.withColumn("Duration", timeDiff)

In [21]:
df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('SR_Flag', 'double'),
 ('Duration', 'bigint')]

In [47]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+--------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Duration|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+--------+
|           HV0003|              B02765|2021-02-05 15:13:57|2021-02-05 15:30:25|          71|          89|   null|     988|
|           HV0003|              B02866|2021-02-04 11:40:47|2021-02-04 11:46:35|          20|         174|   null|     348|
|           HV0003|              B02872|2021-02-02 13:21:54|2021-02-02 13:36:47|          78|         247|   null|     893|
|           HV0003|              B02872|2021-02-03 22:05:16|2021-02-03 22:14:52|          75|         262|   null|     576|
|           HV0003|              B02888|2021-02-02 16:41:27|2021-02-02 17:06:27|         138|          88|   null|    1500|
|       

In [84]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Duration: long (nullable = true)



In [48]:
df.select("pickup_datetime", "Duration").describe().show()

+-------+------------------+
|summary|          Duration|
+-------+------------------+
|  count|          11613942|
|   mean|1010.6961042168111|
| stddev| 672.4338059283223|
|    min|                 0|
|    max|             75540|
+-------+------------------+



In [49]:
E= spark.sql("""
SELECT
    pickup_datetime,
    Duration
FROM
    Amine_data_homework
WHERE
    Duration = '75540'

""").show()

+-------------------+--------+
|    pickup_datetime|Duration|
+-------------------+--------+
|2021-02-11 13:40:44|   75540|
+-------------------+--------+



In [46]:
#spark.sql("""
#SELECT

 #   max(Duration) AS maxdur
#FROM
 #   Amine_data_homework

#""").show()

+------+
|maxdur|
+------+
| 75540|
+------+



In [72]:
new_df = df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID','Duration') \
    .show()

+-----------+------------+------------+------------+--------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|Duration|
+-----------+------------+------------+------------+--------+
| 2021-02-03|  2021-02-03|         181|         144|    1082|
| 2021-02-04|  2021-02-04|          35|         197|    2197|
| 2021-02-05|  2021-02-05|         112|         255|     275|
| 2021-02-01|  2021-02-01|         228|          74|    2111|
| 2021-02-05|  2021-02-05|         141|         141|     425|
| 2021-02-05|  2021-02-05|           3|          51|     853|
| 2021-02-05|  2021-02-05|         241|         174|     722|
| 2021-02-05|  2021-02-05|         209|          68|    1404|
| 2021-02-04|  2021-02-04|         231|         249|    1189|
| 2021-02-04|  2021-02-04|         249|         140|    2091|
| 2021-02-03|  2021-02-03|         162|         244|    1264|
| 2021-02-03|  2021-02-03|           7|         256|    1022|
| 2021-02-03|  2021-02-03|         141|         146|     565|
| 2021-0

In [74]:
new_df.registerTempTable('Amine_data_homework1')

AttributeError: 'NoneType' object has no attribute 'registerTempTable'

In [75]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+--------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Duration|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+--------+
|           HV0003|              B02765|2021-02-05 15:13:57|2021-02-05 15:30:25|          71|          89|   null|     988|
|           HV0003|              B02866|2021-02-04 11:40:47|2021-02-04 11:46:35|          20|         174|   null|     348|
|           HV0003|              B02872|2021-02-02 13:21:54|2021-02-02 13:36:47|          78|         247|   null|     893|
|           HV0003|              B02872|2021-02-03 22:05:16|2021-02-03 22:14:52|          75|         262|   null|     576|
|           HV0003|              B02888|2021-02-02 16:41:27|2021-02-02 17:06:27|         138|          88|   null|    1500|
|       

In [87]:
spark.sql("""
SELECT
    pickup_datetime,
    count(*)
FROM
    Amine_data_homework1
WHERE
    pickup_datetime BETWEEN  '2021-02-15 00:00:00' AND '2021-02-15 23:59:59'
GROUP BY 
    pickup_datetime
""").show()

+-------------------+--------+
|    pickup_datetime|count(1)|
+-------------------+--------+
|2021-02-15 14:27:20|       2|
|2021-02-15 06:56:28|       4|
|2021-02-15 15:43:10|       9|
|2021-02-15 11:48:51|       7|
|2021-02-15 10:24:47|       6|
|2021-02-15 15:41:28|      10|
|2021-02-15 09:15:13|       4|
|2021-02-15 00:11:36|       5|
|2021-02-15 14:19:53|       5|
|2021-02-15 12:13:39|       4|
|2021-02-15 12:45:31|       8|
|2021-02-15 07:18:21|       1|
|2021-02-15 00:53:12|       5|
|2021-02-15 22:04:50|       8|
|2021-02-15 15:35:33|       4|
|2021-02-15 08:44:08|       7|
|2021-02-15 06:40:59|       6|
|2021-02-15 03:19:21|       1|
|2021-02-15 11:50:50|       7|
|2021-02-15 19:11:32|       6|
+-------------------+--------+
only showing top 20 rows



In [91]:
spark.sql("""
SELECT
    count(*)
FROM
    Amine_data_homework1
WHERE
    pickup_datetime BETWEEN  '2021-02-15 00:00:00' AND '2021-02-15 23:59:59'

""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+

